# Diabetes class 구현

In [5]:
import numpy as np
from datetime import datetime

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

In [17]:
import numpy as np
from datetime import datetime

class Diabetes:
    def __init__(self,name,xdata,tdata,i_node,h1_node,o_node,learning_rate,iteration_count):
        self.name = name
        self.xdata = xdata
        if xdata.ndim == 1:
            self.xdata = xdata.reshape(-1,1)
            self.tdata = tdata.reshape(-1,1)
        elif xdata.ndim == 2:
            self.xdata = xdata
            self.tdata = tdata.reshape(-1,1)
        self.i_node = i_node
        self.h1_node = h1_node
        self.o_node = o_node
        self.learning_rate = learning_rate
        self.iteration_count = iteration_count
        
        self.W2 = np.random.rand(self.i_node,self.h1_node)
        self.b2 = np.random.rand(self.h1_node)
        
        self.W3 = np.random.rand(self.h1_node,self.o_node)
        self.b3 = np.random.rand(self.o_node)
    
    def loss_func(self):
        delta = 1e-7
        
        z2 = np.dot(self.xdata, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2,self.W3) + self.b3
        y = a3 = sigmoid(z3) 
        
        return -np.sum(self.tdata * np.log(y + delta) + (1 - self.tdata) * np.log((1 - y) + delta))
    
    def predict(self,test_xdata):
        z2 = np.dot(test_xdata, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2,self.W3) + self.b3
        y = a3 = sigmoid(z3)
        
        if y >= 0.5:
            result = 1
        else:
            result = 0
        
        return y, result
    
    def accuracy(self,test_xdata,test_tdata):
        matched_list = []
        not_matched_list = []
        index_label_prediction_list = []
        for index in range(len(test_xdata)):
            (real_val, logical_val) = self.predict(test_xdata[index])
            if logical_val == test_tdata[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
            T_data = test_tdata[index]
            index_label_prediction_list.append([index,T_data,logical_val])

        print("accuracy => ", len(matched_list) / len(test_xdata))
        return matched_list, not_matched_list, index_label_prediction_list
    
    def train(self):
        f = lambda x : self.loss_func()
        strat_time = datetime.now()
        for step in range(self.iteration_count):
            self.W2 -= self.learning_rate * numerical_derivative(f,self.W2)
            self.b2 -= self.learning_rate * numerical_derivative(f,self.b2)
            
            self.W3 -= self.learning_rate * numerical_derivative(f,self.W3)
            self.b3 -= self.learning_rate * numerical_derivative(f,self.b3)
            
            if (step % 400) == 0:
                print("step = ", step, "error_val = ", self.loss_func())
        end_time = datetime.now()
        print("")
        print("elasped time: ", end_time - strat_time)

In [18]:
load_data = np.loadtxt('./diabetes (1).csv',delimiter=',', dtype = np.float32)
x_data = load_data[0:500,0:-1]
t_data = load_data[0:500,[-1]]

test_x_data = load_data[501:,0:-1]
test_t_data = load_data[501:,[-1]]

#데이터 차원 및 shape 확인
print("x_data.ndim = ", x_data.ndim,", x_data.shape = ", x_data.shape)
print("t_data.ndim = ", t_data.ndim,", t_data.shape = ", t_data.shape)

x_data.ndim =  2 , x_data.shape =  (500, 8)
t_data.ndim =  2 , t_data.shape =  (500, 1)


In [15]:
i_node = x_data.shape[1]
h1_node = 2
o_node = t_data.shape[1]

lr = 1e-3
iter_count = 10001

obj = Diabetes("Diabetes",x_data,t_data,i_node,h1_node,o_node,lr,iter_count)
obj.train()




step =  0 error_val =  345.33625296901636
step =  400 error_val =  252.6510485779644
step =  800 error_val =  245.91983008478226
step =  1200 error_val =  244.77191449649752
step =  1600 error_val =  244.1521759962049
step =  2000 error_val =  243.64553655412647
step =  2400 error_val =  243.17071917113623
step =  2800 error_val =  242.73666875205134
step =  3200 error_val =  242.35334218969678
step =  3600 error_val =  242.01237343562036
step =  4000 error_val =  241.69206214592435
step =  4400 error_val =  241.35798939790016
step =  4800 error_val =  240.95884063696172
step =  5200 error_val =  240.44993370678696
step =  5600 error_val =  239.85891813583356
step =  6000 error_val =  239.26592990248363
step =  6400 error_val =  238.72574433630558
step =  6800 error_val =  238.25387146587227
step =  7200 error_val =  237.8392699836998
step =  7600 error_val =  237.45930811134687
step =  8000 error_val =  237.0896925275495
step =  8400 error_val =  236.70780073945525
step =  8800 error_

In [19]:
(matched_list,not_not_matched_list, index_label_prediction_list) = obj.accuracy(test_x_data,test_t_data)
print(index_label_prediction_list)

accuracy =>  0.813953488372093
[[0, array([1.], dtype=float32), 1], [1, array([1.], dtype=float32), 1], [2, array([0.], dtype=float32), 1], [3, array([1.], dtype=float32), 1], [4, array([1.], dtype=float32), 1], [5, array([1.], dtype=float32), 1], [6, array([1.], dtype=float32), 1], [7, array([0.], dtype=float32), 0], [8, array([0.], dtype=float32), 0], [9, array([1.], dtype=float32), 1], [10, array([1.], dtype=float32), 1], [11, array([1.], dtype=float32), 1], [12, array([1.], dtype=float32), 1], [13, array([1.], dtype=float32), 1], [14, array([0.], dtype=float32), 0], [15, array([1.], dtype=float32), 1], [16, array([1.], dtype=float32), 1], [17, array([1.], dtype=float32), 1], [18, array([1.], dtype=float32), 1], [19, array([1.], dtype=float32), 1], [20, array([1.], dtype=float32), 1], [21, array([1.], dtype=float32), 1], [22, array([1.], dtype=float32), 1], [23, array([1.], dtype=float32), 1], [24, array([1.], dtype=float32), 1], [25, array([1.], dtype=float32), 1], [26, array([0.],